In [1]:
using Pkg
Pkg.instantiate()
Pkg.precompile()

Precompiling project...
┌ Info: Precompiling LimberJack [6b86205d-155a-4b14-b82d-b6a149ea78f2]
└ @ Base loading.jl:1260


In [2]:
using QuadGK
using Interpolations
using OrdinaryDiffEq

In [3]:
include("/home/jaimerz/PhD/LimberJack.jl/src/core.jl")

Cl (generic function with 1 method)

In [4]:
cosmoPar = CosmoPar(0.27, 0.049, 0.67, 0.96, 0.81)
cosmo = Cosmology(cosmoPar);

┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#51 at convenience-constructors.jl:7 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:7
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9


In [15]:
z_arr = [0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,  1]
b_arr = [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]
nz = [[0.0, 0.1, 0.2, 0.30000000000000004,
       0.4, 0.5, 0.6000000000000001, 0.7000000000000001,
       0.8, 0.9, 1.0],
      [0.0, 0.09549150281252627, 0.3454915028125263,
       0.6545084971874737, 0.9045084971874736, 1.0,
       0.9045084971874736, 0.6545084971874737, 0.34549150281252644,
       0.09549150281252634, 0]]
clustering_t = [0.00000000e+00, 4.48526950e-05, 1.71324904e-04, 3.43709458e-04,
        5.04105075e-04, 5.92283813e-04, 5.69713631e-04, 4.38454138e-04,
        2.46077204e-04, 7.22670075e-05, 1.03428840e-10]
lensing_t = [0.00000000e+00, 8.46720571e-06, 1.23677281e-05, 1.23488595e-05,
        9.80438814e-06, 6.34013396e-06, 3.25773350e-06, 1.22774325e-06,
        2.76337744e-07, 1.90522187e-08, 0.00000000e+00]
CMBlensing_t = [0.00000000e+00, 1.11161519e-05, 2.29070311e-05, 3.51531866e-05,
        4.76791214e-05, 6.03482340e-05, 7.30566167e-05, 8.57272238e-05,
        9.83037415e-05, 1.10746285e-04, 1.23027399e-04];

In [6]:
qg = clustering_kernel(cosmo, 2, nz)
clustering_jl = [qg(z, 1) for z in z_arr]

11-element Array{Array{Float64,1},1}:
 [0.0]
 [4.4548853250678867e-5]
 [0.0001689229691343614]
 [0.0003365196973273108]
 [0.0004903265026169991]
 [0.0005726265297483702]
 [0.0005478020119701433]
 [0.00041953171502033624]
 [0.00023443530416709528]
 [6.858372281020839e-5]
 [0.0]

In [7]:
clustering_jl./clustering_t

11-element Array{Array{Float64,1},1}:
 [NaN]
 [0.9932257861133844]
 [0.9859802351581145]
 [0.9790818654961506]
 [0.9726672611201129]
 [0.9668110408892269]
 [0.9615392403523927]
 [0.9568428682963787]
 [0.9526900515624166]
 [0.9490322788059045]
 [0.0]

In [8]:
lensing_jl = [lensing_kernel(cosmo, z, nz) for z in z_arr]

11-element Array{Float64,1}:
 0.0
 3.596894945051763e-6
 5.293478563722389e-6
 5.334025411814407e-6
 4.284513928527437e-6
 2.8157314623322474e-6
 1.481964424606995e-6
 5.838480631301325e-7
 1.4738546267254347e-7
 1.612414943703329e-8
 0.0

In [9]:
lensing_jl./lensing_t

11-element Array{Float64,1}:
 NaN
   0.42480306588084094
   0.4280073527588619
   0.43194478095846883
   0.43699962377534324
   0.444112297957226
   0.4549065860074174
   0.47554573248937226
   0.5333526305134179
   0.84631347618497
 NaN

In [17]:
qCMBL = CMBlensing_kernel(cosmo)
CMBlensing_jl = [qCMBL(z, 1) for z in z_arr]

11-element Array{Float64,1}:
 0.0
 9.74556879098948e-6
 2.0790285828252183e-5
 3.299630065572857e-5
 4.624056149130983e-5
 6.040342348446177e-5
 7.538477899374835e-5
 9.108846896932182e-5
 0.00010743808478098922
 0.0001243594093339025
 0.0001417960411746628

In [21]:
CMBlensing_jl./CMBlensing_t

11-element Array{Float64,1}:
 NaN
   0.8767034562553503
   0.9075940805027406
   0.938643231158127
   0.9698283050012292
   1.0009145169759528
   1.0318679183202355
   1.0625384204885662
   1.09291958924055
   1.1229217244975982
   1.1525566038721407

In [11]:
CMBlensing_jl./CMBlensing_t

UndefVarError: [91mUndefVarError: CMBlensing_jl not defined[39m

In [12]:
qgamma = shear_kernel(cosmo, nz)
qgamma(1, [3, 5])

2-element Array{Float64,1}:
 0.0
 0.0

In [13]:
qk = convergence_kernel(cosmo, BOSS_nz)
qk(1, [3, 5])

UndefVarError: [91mUndefVarError: BOSS_nz not defined[39m

In [14]:
Cl(cosmo, [1, 2], qg, qk)

UndefVarError: [91mUndefVarError: qk not defined[39m